In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/dp/5vg_3lhn6tz7w33s1288bclr0000gn/T/ipykernel_87838/2729103167.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/dp/5vg_3lhn6tz7w33s1288bclr0000gn/T/ipykernel_87838/2729103167.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [4]:
print(vectordb._collection.count())

228


In [5]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [6]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/var/folders/dp/5vg_3lhn6tz7w33s1288bclr0000gn/T/ipykernel_87838/60327743.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0)


### RetrievalQA chain

In [7]:
from langchain.chains import RetrievalQA

In [8]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [9]:
result = qa_chain({"query": question})

/var/folders/dp/5vg_3lhn6tz7w33s1288bclr0000gn/T/ipykernel_87838/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [10]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. In addition to these main topics, there will be discussions covering extensions of the material taught in the main lectures.'

### Prompt

In [11]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [12]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [13]:
question = "Is probability a class topic?"

In [14]:
result = qa_chain({"query": question})

In [15]:
result["result"]

'Probability is assumed to be a prerequisite for the class, so it will not be a main topic covered in the course. Thanks for asking!'

In [16]:
result["source_documents"][0]

Document(metadata={'author': '', 'creationdate': '2008-07-11T11:25:23-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:23-07:00', 'page': 4, 'page_label': '5', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': 'MachineLearning-Lecture01.pdf', 'title': '', 'total_pages': 22}, page_content="of this class will not be very programming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  \nI also assume familiarity with basic probability and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what random variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort o

### RetrievalQA chain types

In [17]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [18]:
result = qa_chain_mr({"query": question})

In [19]:
result["result"]

'Yes, probability is a class topic.'

If you wish to experiment on the `LangSmith platform` (previously known as LangChain Plus):

 * Go to [LangSmith](https://www.langchain.com/langsmith) and sign up
 * Create an API key from your account's settings
 * Use this API key in the code below   
 * uncomment the code  
 Note, the endpoint in the video differs from the one below. Use the one below.

In [20]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..." # replace dots with your api key

In [21]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'No, probability is assumed to be a prerequisite for the class, not a topic covered in the class itself.'

In [22]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'Yes, probability is a topic that will be covered in the class. The instructor assumes familiarity with basic probability and statistics, as well as basic linear algebra. The course may involve some programming, mostly in MATLAB or Octave, but it will not be heavily programming intensive. The class will also include review sections to refresh students on prerequisite concepts if needed, such as random variables, expectations, variances, matrices, vectors, matrix multiplication, matrix inverses, and eigenvectors. Students who have taken undergraduate statistics and linear algebra courses should have sufficient background knowledge for the class. The instructor will also provide additional support and review sessions for those who may need a refresher on these topics.'

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [24]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic for the course being described. The instructor assumes familiarity with basic probability and statistics for the class.'

In [25]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites of basic probability and statistics, as well as basic linear algebra, are needed for this class because the course assumes familiarity with these subjects. Understanding random variables, expectations, variances, matrices, vectors, matrix multiplication, and matrix inverses are fundamental concepts that will be used throughout the course. These prerequisites are essential for students to grasp the material covered in the class effectively.'